### Wykład 3 21.10.2020

#### Funkcje wyższego rzędu

Funkcja wyższego rzędu
to funkcja, która zwraca i/lub przyjmuje jako argument(y) inne funkcje

In [ ]:
sumWith :: Num a => (a -> a) -> [a] -> a
sumWith _ [] = 0
sumWith f (x:xs) = f x + sumWith f xs

In [ ]:
sum, sumSqr, sumCub, sumAbs :: [Integer] -> Integer

sum = sumWith (\e -> e) -- sumWith (id)
sumSqr = sumWith (\e -> e ^ 2) -- sumWith (^2)
sumCub = sumWith (\e -> e ^ 3) -- sumWith (^3)
sumAbs = sumWith (\e -> abs e) -- sumWith (abs)

sumWith (\e -> e^2) [1,2,3]


#### Funkcje Anonimowe (Lambda-wyrażenia)

In [ ]:
(\x y -> x + y) 1 2

#### Funkcje jako wyniki:

In [ ]:
df :: (Double -> Double) -> (Double -> Double)
df f = \x -> (f (x + h) - f (x - h)) / (2 * h)
    where h = 1e-8
    
absErr :: Num a => (t -> a) -> (t -> a) -> t -> a
absErr fExact fApprox x=abs (fExact x - fApprox x)

xs = [0,0.2..1.0]
zip xs (map (absErr (\x -> 2 * x) (df (\x -> x^2))) xs)

In [ ]:
absErr (\x -> 2 * x) (df (\x -> x^2)) :: Double -> Double
-- Jest wywołana z ? argumentami

#### Domknięcie funkcji (closure)

In [ ]:
df :: (Double -> Double) -> Double -> (Double -> Double)
df f h = \x -> (f (x + h) - f (x - h)) / (2 * h)
dfSqrXDx = df (\x -> x^2) 1e-8
dfSqrXDx 1

### Operatory (.) (złożenie funkcji) i \dollar (obliczenie wartości/’aplikacji’ funkcji)

In [ ]:
-- (f ◦g)(x) = f(g(x)) Złożenie w matematyce

(.) :: (b -> c) -> (a -> b) -> a -> c
f . g = \x -> f (g x)

f x = 2 * x
g x = x ^ 2

f_o_g x = (f . g) x -- 2 * x^2
g_o_f = g . f -- 4 * x^2
f_o_g 3 -- 18
g_o_f 3 -- 36

In [ ]:
-- UWAGA!  $ priorytet 0, prawostronna łączność
($) :: (a -> b) -> a -> b
f $ x = f x

f = \x y -> x^2 + y^2
f 1 2 + 3 -- 8
f 1 $ 2 + 3 -- 26

-- Uwaga
-- istnieje też operator $! (f $! x) – wersja ’strict’ operatora $

### Wzorzec ‘Collection Pipeline’

#### Funkcje wyższego rzędu: ﬁlter

In [ ]:
filter :: (a -> Bool) -> [a] -> [a]
filter _ [] = []
filter p (x:xs)
    | p x = x : filter p xs
    | otherwise = filter p xs
    
filter (\x -> x < 5 && x > 1) [1..10] -- [2,3,4]
filter (<5) [1..10] -- [1,2,3,4]
filter even [1..10] -- [2,4,6,8,10]

In [ ]:
 filter (\s -> length s == 2) ["a", "aa", "aaa", "b", "bb"]
["aa","bb"]

In [ ]:
filter (\(x,y) -> x > y) [(1,2), (2,2), (2,1), (2,2), (3,2)]
[(2,1),(3,2)]

In [ ]:
-- ﬁlter and list comprehension
filter p xs = [x | x <- xs, p x]

#### Funkcje wyższego rzędu: map

In [ ]:
map :: (a -> b) -> [a] -> [b]
map _ [] = []
map f (x:xs) = f x : map f xs

map (\e -> e^2) [1..5] -- [1,4,9,16,25]
map (1+) [1..5] -- [2,3,4,5,6]
map ($ 2) [(^2), (^3), (^4), (^5)] -- [4,8,16,32]

In [ ]:
 map (\e -> (e, length e)) ["My","name","is","Inigo","Montoya"]
[("My",2),("name",4),("is",2),("Inigo",5),("Montoya",7)]

In [ ]:
-- map and list comprehension
map f xs = [f x | x <- xs]

#### Funkcje wyższego rzędu: foldr [reduce]

In [ ]:
-- Częsty schemat rekursji*
f [] = z
f (x:xs) = x `op` (f xs)
-- sum: z = 0, op = +
-- prod: z = 1, op = *

In [ ]:
foldr :: (a -> b -> b) -> b -> [a] -> b
foldr f z [] = z
foldr f z (x:xs) = f x (foldr f z xs)

foldr (+) 0 [1..5] -- 15
foldr (*) 1 [1..5] -- 120
foldr (||) False [False, False, True, False] -- True
foldr (&&) True [False, False, True, False] -- False
foldr (\_ n -> 1 + n) 0 [1..10] -- 10
foldr (\x xs -> xs ++ [x]) [] [1..5] -- [5,4,3,2,1]

In [ ]:
-- Rekursja z akumulatorem” [z ∼ acc]
f :: Num t => t -> [t] -> t
f z [] = z -- z, op jak dla foldr
f z (x:xs) = f (x `op` z) xs

In [ ]:
foldl :: (b -> a -> b) -> b -> [a] -> b
foldl f z [] = z
foldl f z (x:xs) = foldl f (f z x) xs

-- Funkcje pokrewne:

foldl1 :: Foldable t => (a -> a -> a) -> t a -> a
foldr1 :: Foldable t => (a -> a -> a) -> t a -> a
foldl' :: Foldable t => (b -> a -> b) -> b -> t a -> b

#### Funkcje: zip, unzip, zipWith

In [ ]:
zipWith :: (a -> b -> c) -> [a] -> [b] -> [c]
zipWith _ [] _ = []
zipWith _ _ [] = []
zipWith f (x:xs) (y:ys) = f x y : zipWith f xs ys

In [ ]:
zip :: [a] -> [b] -> [(a,b)]
zip _ [] = []
zip [] _ = []
zip (x:xs) (y:ys) = (x,y) : zip xs ys

unzip :: [(a,b)] -> ([a],[b])
unzip [] = ([],[])
unzip ((x,y) : xys) = (x:xs, y:ys
    where (xs, ys) = unzip xys

In [ ]:
zip [1,2,3] ['a','b'] -- [(1,'a'),(2,'b')]
unzip [(1,'a'),(2,'b')] -- ([1,2],"ab")
unzip (zip [1,2,3] ['a','b']) -- ([1,2],"ab")
zipWith (+) [1..5] [5,4..1] -- [6,6,6,6,6]

#### Funkcja concat [ﬂatten]

In [ ]:
concat :: [[a]] -> [a]
concat = foldr (++) []

concat [[1,2],[3,4]]
(concat . concat) [ [[1,2], [3,4]] , [[5,6], [7,8]] ]

#### Wzorzec ’Collection pipeline’

In [ ]:
capitalize :: [Char] -> [Char] -- Data.Char, Data.List needed
capitalize [] = []
capitalize (x:xs) = toUpper x : (map toLower xs)

formatStr s = 
    foldr1 (\w s -> w ++ " " ++ s) .
    map capitalize .
    filter (\x -> length x > 1) $
    words s
                
formatStr "tomasz t bogdan anna Jerzy j maria"
"Tomasz Bogdan Anna Jerzy Maria"